# Homework 5: Model-based Relative Entropy Stochastic Search (15 Pts)

All homeworks are self-contained. They can be completed in their respective notebooks.
To edit and re-run code, you can therefore simply edit and restart the code cells below.
There is a timeout of about ~12 hours with Colab while it is active (and less if you close your browser window).
This file should automatically be synced with your Google Drive. We also save all recordings and logs in it by default so that you won't lose your work in the event of an instance timeout.
 However, you will need to re-mount your Google Drive and re-install packages with every new instance.

In [ ]:
# Your work will be stored in a folder called `drl_ws21` by default to prevent Colab
# instance timeouts from deleting your edits.
# We do this by mounting your google drive on the virtual machine created in this colab
# session. For this, you will likely need to sign in to your Google account and copy a
# passcode into a field below

import os
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
# Create paths in your google drive
DRIVE_PATH = '/content/gdrive/My\ Drive/drl_ws21'
DRIVE_PYTHON_PATH = DRIVE_PATH.replace('\\', '')
if not os.path.exists(DRIVE_PYTHON_PATH):
    % mkdir $DRIVE_PATH

# the space in `My Drive` causes some issues,
# make a symlink to avoid this
SYM_PATH = '/content/drl_ws21'
if not os.path.exists(SYM_PATH):
    !ln -s $DRIVE_PATH $SYM_PATH
% cd $SYM_PATH

In [ ]:
# Install **python** and **system** packages

# install required python dependencies
!pip install matplotlib numpy tqdm scipy nlopt

We start by importing all the necessary python modules and defining some helper
functions which you do not need to change. Still, make sure you are aware of
what they do.

In [ ]:
# Imports and utility
# Progress bar

import os
import time
import tqdm
import nlopt
import numpy as np
from typing import *
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from scipy.stats import multivariate_normal

# Set random seeds
np.random.seed(0)

class ProgressBar:
    def __init__(self, num_iterations: int, verbose: bool = True):
        if verbose:  # create a nice little progress bar
            self.scalar_tracker = tqdm.tqdm(total=num_iterations, desc="Scalars", bar_format="{desc}",
                                            position=0, leave=True)
            progress_bar_format = '{desc} {n_fmt:' + str(
                len(str(num_iterations))) + '}/{total_fmt}|{bar}|{elapsed}<{remaining}'
            self.progress_bar = tqdm.tqdm(total=num_iterations, desc='Iteration', bar_format=progress_bar_format,
                                          position=1, leave=True)
        else:
            self.scalar_tracker = None
            self.progress_bar = None

    def __call__(self, _steps: int = 1, **kwargs):
        if self.progress_bar is not None:
            formatted_scalars = {key: "{:.3e}".format(value[-1] if isinstance(value, list) else value)
                                 for key, value in kwargs.items()}
            description = ("Scalars: " + "".join([str(key) + "=" + value + ", "
                                                  for key, value in formatted_scalars.items()]))[:-2]
            self.scalar_tracker.set_description(description)
            self.progress_bar.update(_steps)

    def update(self, increment: int):
        if self.progress_bar is not None:
            self.progress_bar.update(increment)

# specify the path to save the recordings of this run to.
data_path = '/content/drl_ws21/exercise_5'
data_path = os.path.join(data_path, time.strftime("%d-%m-%Y_%H-%M"))
if not (os.path.exists(data_path)):
    os.makedirs(data_path)

# this function will automatically save your figure into your google drive folder (if correctly mounted!)
def save_figure(save_name: str) -> None:
    assert save_name is not None, "Need to provide a filename to save to"
    plt.savefig(os.path.join(data_path, save_name + ".png"))


def plot_metrics(metrics: Dict[str, List[float]]):
    """
    Plots various metrics recorded during training
    :param metrics:
    :return:
    """
    if len(metrics) > 0:
        plt.clf()
        plt.figure(figsize=(16, 9))
        for position, (key, value) in enumerate(metrics.items()):
            plt.subplot(len(metrics), 1, position + 1)
            plt.plot(range(len(value)), np.array(value))
            plt.ylabel(key.title())
        plt.xlabel("Recorded Steps")
        plt.tight_layout()
        save_figure(f"training_metrics")
        plt.clf()
        plt.close()


# **Model-based Relative Entropy Stochastic Search (15 Pts)**

This exercise is about Model-based Relative Entropy Stochastic Search (MORE), a gradient free policy search algorithm
for blackbox function optimization.
MORE makes use of a Gaussian search distribution with a full covariance matrix, and iteratively updates this distribution
by
* drawing samples from it
* evaluating the samples on the target function
* fitting a quadratic surrogate model on the samples and their targets
* using this model to update the search distribution in closed form under entropy and KL constraints

The algorithm is comparatively involved and contains a lot of details and complex expressions. As such, this exercise
will not dive into the core algorithm too deeply. Instead, we will focus on small, individual parts of it
as well as on some pen&paper tasks. You are of course still incentivized to look at it in more detail.

For now, we will start by defining our task and then the individual parts, namely the Gaussian and the surrogate model
that are needed for MORE.

## Point Reacher
Our task for this exercise is a planar point reaching task. A robot arm starting at position $(0,0)^T$ with $n$ links
of length $1$ is tasked to reach a point at positions $(0.7*n, 0)^T$.
The action space consist of a continuous angle for each of the $n$ joints.
To incentivize smooth solutions, there is an additional penalty term on the squared angles.

You do **not** need to adapt the code for this task.

In [ ]:
class PointReacher:

    def __init__(self, num_links: int, likelihood_std: float, smoothness_prior_std: Union[np.array, List[float]]):
        """
        Initialization of a simple point reacher task, where the goal is to reach a point (0, num_links*0,7) using
        a robot arm with num_links joints of length 1. Note that this task does *not* use time-series data, but
        instead evaluates a single angle configuration.
        :param num_links: Number of links of the robot
        :param likelihood_std: The "main" reward (not regarding the smoothness prior) is the closeness to the point/line.
            This reward is represented as a Gaussian with likelihood likelihood_std
        :param smoothness_prior_std: Standard deviation of a zero-mean Gaussian acting in angle space.
            Adds a smoothness prior for each joint, with smaller values leading to smoother solutions.
        """
        self._num_links = num_links
        self.target = [0.7 * num_links, 0]
        self._smoothness_likelihood = multivariate_normal(np.zeros(num_links),
                                                          np.array(smoothness_prior_std) * np.eye(num_links))
        self._target_likelihood = multivariate_normal(self.target, likelihood_std * np.eye(2))

    def reward(self, samples: np.array) -> np.array:
        """
        Calculates the reward for the given angles. Good angle configurations are those that
        * reach the target/have a high target (log) likelihood
        * are smooth/have a high smoothness (log) likelihood
        :param samples: An array of shape (..., num_angles) to evaluate
        :return: An array of shape (...), where each entry corresponds to the reward of the
          corresponding sample. Higher values are better.
        """
        samples = self.angle_normalize(samples)
        end_effector_position = self.forward_kinematic(joint_angles=samples)[..., -1, :]
        target_likelihood = self._target_likelihood.logpdf(end_effector_position[..., 0:])
        smoothness_likelihood = self._smoothness_likelihood.logpdf(samples)
        return np.squeeze(target_likelihood + smoothness_likelihood)

    @staticmethod
    def forward_kinematic(joint_angles: Union[List[np.array], np.array]) -> Union[List[np.array], np.array]:
        """
        Calculates the forward kinematic of the robot by interpreting each input value as an angle

        :param joint_angles: The angles of the joints. Can be of arbitrary shape, as long as the last dimension is over
            the relative angles of the robot. I.e., the shape is (..., angles)
        :return: The positions as an array of shape (..., #angles, 2), where the last dimension is the x and y position of
         each angle.
        """
        angles = np.cumsum(joint_angles, axis=-1)
        pos = np.zeros([*angles.shape[:-1], angles.shape[-1] + 1, 2])
        for i in range(angles.shape[-1]):
            pos[..., i + 1, 0] = pos[..., i, 0] + np.cos(angles[..., i])
            pos[..., i + 1, 1] = pos[..., i, 1] + np.sin(angles[..., i])
        return pos

    @staticmethod
    def angle_normalize(angles: np.array) -> np.array:
        """
        Normalizes the angles to be in [-pi, pi]
        :param angles: Unnormalized input angles
        :return: Normalized angles
        """
        return ((angles + np.pi) % (2 * np.pi)) - np.pi

    def render(self, samples: np.array, iteration: int = 0):
        plt.gca().add_patch(Rectangle(xy=(-0.1, -0.6), width=0.1, height=1.2, facecolor="grey", alpha=1, zorder=100))
        plt.xlabel(r"$x$")
        plt.ylabel(r"$y$")
        axes = plt.gca()
        axes.set_xlim([-0.6 * self._num_links, 1.1 * self._num_links])
        axes.set_ylim([-0.7 * self._num_links, 0.7 * self._num_links])
        axes.set_aspect(aspect="equal")

        rewards = self.reward(samples=samples)
        angles = self.forward_kinematic(joint_angles=samples)
        for reward, configuration in zip(rewards, angles):
            plt.plot(configuration[:, 0], configuration[:, 1], 'o-', markerfacecolor='k', alpha=0.75,
                     label=f"{reward:.3e}")
        plt.scatter(self.target[0], self.target[1], c="r", marker="x", s=100)
        plt.legend(loc="upper left", ncol=2)
        plt.title(f"Point Reacher samples and reward at iteration {iteration}")

## Gaussian

The next code cell defines a Gaussian class containing the utility functions that are needed for the MORE algorithm.
You do **not** need to implement anything here, but should have a rough understanding of what the individual parameters
and functions do.

In [ ]:
class Gaussian:
    """
    A multivariate Gaussian with a full covariance matrix
    """

    def __init__(self, mean: np.array, covariance: np.array):
        if len(mean.shape) < 2:
            mean = np.atleast_2d(mean).reshape([-1, 1])
        self.task_dimension = mean.shape[0]
        self.mean = mean
        self.covariance = covariance

        self.precision = None  # precision of q
        self.chol_prec = None  # cholesky of the precision
        self.natural_mean = None  # canonical mean parameter of q
        self.log_det = None  # log determinant
        self.chol_cov = None  # cholesky of the covariant

        self.update_params(mean, covariance)

        # precompute constant value
        self._log_2_pi_k = self.task_dimension * (np.log(2 * np.pi))

    def update_params(self, mean: np.array, covariance: np.array) -> None:
        if len(mean.shape) < 2:
            mean = np.atleast_2d(mean).reshape([-1, 1])
        self.mean = mean
        self.covariance = covariance

        self.chol_cov = np.linalg.cholesky(self.covariance)
        inv_chol_cov = np.linalg.inv(self.chol_cov)  # cholesky of precision of q
        self.precision = inv_chol_cov.T @ inv_chol_cov
        self.chol_prec = np.linalg.cholesky(self.precision)
        self.natural_mean = self.precision @ self.mean
        self.log_det = 2 * np.sum(np.log(np.diag(self.chol_cov)))

    def sample(self, n_samples: int):
        z = np.random.normal(size=(n_samples, self.task_dimension)).T
        x = self.mean + self.chol_cov @ z
        return x.T

    @property
    def entropy(self) -> float:
        """
        Compute entropy in closed form
        :return:
        """
        return 0.5 * (self.task_dimension + self._log_2_pi_k + self.log_det)

## **TASK 1: Quadratic Surrogate Model** (2+1=3 Points)

MORE uses a quadratic surrogate model to approximately fit the target function in the local area of its search
distribution.
The surrogate model is fit using least-squares regression.

This surrogate is simple enough to allow for a closed-form update of the search distribution, while still allowing
for an expressive feedback that is sufficiently similar to the original (and unknown) reward function.

### Task 1.1: Whitening (2 Points)
The surrogate is fit on samples from the search distribution as inputs, and target function evaluations as targets.
For stability purposes, the samples/inputs are [whitened](https://en.wikipedia.org/wiki/Whitening_transformation)
before the fit, i.e., they are linearly transformed such that their mean is $\mathbf{0}$ and
their covariance the identity matrix $I$. Your first task is to implement this whitening.
Hint: You can use `np.cov(rowvar=False)` to get the covariance of a list of samples,
`np.linalg.cholesky()` for the cholesky decomposition of a covariance matrix and `np.linalg.inv()` to get a matrix inverse
Hint 2: Remember to subtract the mean of the samples

### Task 1.2: Fitting the surrogate (1 Point)
After the data is preprocessed, fitting the surrogate is a comparatively simple least squares regression. To implement
this regression, you can use the `np.linalg.solve()` function.

In [ ]:

class QuadraticModel:
    def __init__(self, dimension: int):
        self.dimension = dimension

        self._quadratic_term = np.eye(self.dimension)
        self._linear_term = np.zeros(shape=(self.dimension, 1))

        self.dim_tri = int(self.dimension * (self.dimension + 1) / 2)
        self.model_dim = 1 + self.dimension + self.dim_tri

        self.square_feat_lower_tri_ind = np.tril_indices(self.dimension)

        self.feature_matrix = None
        self.targets = None

        self._data_mean = None
        self._data_inverse_std = None

    def preprocess_data(self, inputs: np.array, targets: np.array) -> bool:
        if len(targets.shape) < 2:
            targets = targets[:, None]

        inputs = self.whiten(inputs)
        self.feature_matrix = self.quadratic_features(inputs)
        self.targets = (targets - np.mean(targets)) / (np.std(targets, ddof=1) + 1.0e-8)  # normalize targets
        return True

    def whiten(self, input: np.array) -> np.array:
        """
        Whiten the data, i.e., normalize their mean and variance such that the data appears to be drawn from a Gaussian
        N(0,I)
        :param input: Input samples.
        :return: Whitened samples.
        """
        ### TODO ### (2pts)
        ### Your code starts here ###
        data_mean = np.mean(input, axis=0, keepdims=True)
        data_inverse_std = np.linalg.inv(np.linalg.cholesky(np.cov(input, rowvar=False))).T
        input = input - data_mean
        input = input @ data_inverse_std
        ### Your code ends here ###

        # store values for unwhitening later on
        self._data_mean = data_mean
        self._data_inverse_std = data_inverse_std

        return input

    def unwhiten_params(self, quadratic_term, linear_term, constant_term):
        quadratic_term = self._data_inverse_std @ quadratic_term @ self._data_inverse_std.T
        tmp_linear_term = self._data_inverse_std @ linear_term
        linear_term = (self._data_mean @ quadratic_term).T + tmp_linear_term
        constant_term = constant_term + self._data_mean @ (quadratic_term @ self._data_mean.T - tmp_linear_term)

        return quadratic_term, linear_term, constant_term

    def quadratic_features(self, inputs: np.array) -> np.array:
        lin_feat = inputs
        quad_feat = np.transpose((inputs[:, :, None] @ inputs[:, None, :]),
                                 [1, 2, 0])[self.square_feat_lower_tri_ind].T

        feature_matrix = np.hstack([np.ones([inputs.shape[0], 1]), lin_feat, quad_feat])  # c + bx + ax**2
        return feature_matrix

    def fit(self, inputs: np.array, targets: np.array) -> bool:
        """
        Fit the quadratic model on the inputs and targets
        """
        success = self.preprocess_data(inputs, targets)
        if not success:
            return False

        feature_matrix = self.feature_matrix
        targets = self.targets

        regularization_matrix = np.eye(self.model_dim) * 1e-10
        regularization_matrix[0, 0] = 0
        # we add an additional regularization term to the result of the matrix multiplications related to the feature
        # matrix for numerical stability. See solution for ridge regression in the literature

        ### TODO ### (1pts)
        ### Your code starts here ###
        parameters = np.linalg.solve(feature_matrix.T @ feature_matrix + regularization_matrix,
                                     feature_matrix.T @ targets)
        ### Your code ends here ###

        self._quadratic_term, self._linear_term = self.postprocess_params(parameters)

        return True

    def postprocess_params(self, parameters: np.array) -> Tuple[np.array, np.array]:
        quadratic_term = np.zeros((self.dimension, self.dimension))
        triangular_part = parameters[self.dimension + 1:].flatten()
        quadratic_term[self.square_feat_lower_tri_ind] = triangular_part
        quadratic_term = - (quadratic_term + quadratic_term.T)

        linear_term = parameters[1:self.dimension + 1]

        return quadratic_term, linear_term

    @property
    def parameters(self) -> Tuple[np.array, np.array]:
        return self._quadratic_term, self._linear_term

## **Task 2: Updating the natural parameters** (3 Points)

We update the natural parameters of the search distribution using the equations of Slide Set 8, Slide 33. At iteration
k, we update the mean and covariance using

\begin{align*}
\Sigma_{k+1} = \left(\frac{{\eta} \Sigma_{k}^{-1} +  A}{{\eta} + {\omega}}\right)^{-1}
\end{align*}

\begin{align*}
\vec \mu_{k+1} = \Sigma_{k+1} \left(\frac{{\eta} \Sigma_{k}^{-1} \mu_k + a}{{\eta}+ {\omega}} \right)
\end{align*}

where we use $\omega$ instead of the $\kappa$ used in the slides for consistency with the code.
As most of the operations needed for MORE
are in the natural parameter space of the Gaussian, we update the **natural mean** ($=\Sigma^{-1} \mu$) and the
 **precision** ($=\Sigma^{-1}$) rather than the "regular" mean and variance.

Hint: Updating the parameters in natural space simplifies these equations. You only need to implement this simplified
version.

In [ ]:

def new_natural_params(eta: float, omega: float, old_distribution: Gaussian, model: QuadraticModel):
    """
    Compute the new natural parameters (the natural mean and the precision) of the search distribution
    using the optimal dual variables and the old distribution.
    :param eta: Solution for the dual eta
    :param omega: Solution for the dual omega
    :param old_distribution: Old search distribution
    :param model: The quadratic surrogate model that is used to fit the distribution
    :return:
    """
    ### TODO ### (3pts)
    ### Your code starts here ###
    quadratic_parameters, linear_parameters = model.parameters
    natural_mean = (eta * old_distribution.natural_mean + linear_parameters) / (eta + omega)  # natural mean of pi
    precision = (eta * old_distribution.precision + quadratic_parameters) / (eta + omega)  # precision of pi
    ### Your code ends here ###

    return natural_mean, precision

## MORE
### **Task 3: Deriving the MORE equations** (9 Points)
Here we are going to derive the primal solution of the optimization problem underlying the Model-based Relative Entropy Stochastic Search Approach (MORE).
Recall that the MORE optimization problem is given as
\begin{align}
    \underset{\boldsymbol{\omega}}{\textrm{argmax}} \int p_{\boldsymbol{\omega}}(\boldsymbol{\theta})g(\boldsymbol{\theta}) d\boldsymbol{\theta} \quad \textrm{s.t.} \quad \textrm{KL}(p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) || p_{\textrm{old}}(\boldsymbol{\theta})) \leq \epsilon, \quad \textrm{H}(p_{\textrm{old}}(\boldsymbol{\theta})) - \textrm{H}(p_{\boldsymbol{\omega}}(\boldsymbol{\theta})) \leq \gamma, \quad \int p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) d \boldsymbol{\theta} = 1.
\end{align}
As a first simplification we can rewrite this objective as
\begin{align}
    \underset{\boldsymbol{\omega}}{\textrm{argmax}} \int p_{\boldsymbol{\omega}}(\boldsymbol{\theta})g(\boldsymbol{\theta}) d\boldsymbol{\theta} \quad \textrm{s.t.} \quad \textrm{KL}(p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) || p_{\textrm{old}}(\boldsymbol{\theta})) \leq \epsilon, \quad \textrm{H}(p_{\boldsymbol{\omega}}(\boldsymbol{\theta})) \geq \beta, \quad \int p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) d \boldsymbol{\theta}=1,
\end{align}
with $\beta = \textrm{H}(p_{\textrm{old}}(\boldsymbol{\theta})) - \gamma$.

Denoting the Lagrangian multipliers for the KL and Entropy constraint by $\eta$ and $\kappa$ respectivly, show that
\begin{align}
    p_{\boldsymbol{\omega}^*}(\boldsymbol{\theta}) \propto p_{\textrm{old}}(\boldsymbol{\theta})^{\frac{\eta}{\eta + \kappa}} \exp\left( \dfrac{g(\boldsymbol{\theta})}{\eta + \kappa} \right).
\end{align}
Note that $p_{\boldsymbol{\omega}^*}(\boldsymbol{\theta})$ is the optimal solution to the optimization
problem depending on the duals $\eta$ and $\kappa$

Hint: The parameter $\kappa$ used in the equations is called $\omega$ in the code.
The $\omega$ parameter in the equations does not have a direct code equivalent.

### Solution
The solution is more or less (without the entropy constraint) in the Trust Region slides.
To compute the update formulas and the dual we start by deriving the Lagrangian:
\begin{align}
\mathcal{L}(p_{\boldsymbol{\omega}}(\boldsymbol{\theta}), \eta, \kappa, \lambda) \nonumber& \\
= \int p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) g(\boldsymbol{\theta})d\theta + \eta \left(\epsilon - \textrm{KL}(p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) || p_{\textrm{old}}(\boldsymbol{\theta})) \right) + \kappa \left(\textrm{H}(p_{\boldsymbol{\omega}}(\boldsymbol{\theta})) - \beta \right) + \lambda \left( 1 - \int p_{\boldsymbol{\omega}}(\boldsymbol{\theta})d\theta \right) \\
= \eta\epsilon - \kappa\beta + \lambda  + \int p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) g(\boldsymbol{\theta})d\theta - \eta \textrm{KL}(p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) || p_{\textrm{old}}(\boldsymbol{\theta})) + \kappa \textrm{H}(p_{\boldsymbol{\omega}}(\boldsymbol{\theta})) - \lambda \int p_{\boldsymbol{\omega}}(\boldsymbol{\theta})d\theta \nonumber \\
= \eta\epsilon - \kappa\beta + \lambda  + \int p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) \left( g(\boldsymbol{\theta}) - \eta\left(\log p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) - \log p_{\textrm{old}}(\boldsymbol{\theta}) \right) - \kappa \log p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) - \lambda \right) d\theta \nonumber \\
= \eta\epsilon - \kappa\beta + \lambda  + \int p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) \left( g(\boldsymbol{\theta}) - (\eta + \kappa) \log p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) + \eta \log p_{\textrm{old}}(\boldsymbol{\theta}) - \lambda \right) d\theta. \label{eq:marg:lagrangian_simplified}
\end{align}

Derivative w.r.t. $p_{\boldsymbol{\omega}}(\boldsymbol{\theta})$

\begin{align}
\dfrac{\partial \mathcal{L}}{\partial p_{\boldsymbol{\omega}}(\boldsymbol{\theta})} &= \int \dfrac{\partial}{\partial p_{\boldsymbol{\omega}}(\boldsymbol{\theta})} p_{\boldsymbol{\omega}}(\boldsymbol{\theta})  \left( g(\boldsymbol{\theta}) - (\eta + \kappa) \log p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) + \eta \log p_{\textrm{old}}(\boldsymbol{\theta}) - \lambda \right) d\theta \\
&=\int  \left(  p_{\boldsymbol{\omega}}(\boldsymbol{\theta})\dfrac{-(\eta + \kappa)}{p_{\boldsymbol{\omega}}(\boldsymbol{\theta})} +  g(\boldsymbol{\theta}) - (\eta + \kappa) \log p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) + \eta \log p_{\textrm{old}}(\boldsymbol{\theta}) - \lambda \right) d\theta \\
&=\int  -(\eta + \kappa + \lambda) +  g(\boldsymbol{\theta}) - (\eta + \kappa) \log p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) + \eta \log p_{\textrm{old}}(\boldsymbol{\theta}) d\theta.
\end{align}

Setting the derivative to $0$

\begin{align*}
& -(\eta + \kappa + \lambda) +  g(\boldsymbol{\theta}) - (\eta + \kappa) \log p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) + \eta  \log p_{\textrm{old}}(\boldsymbol{\theta}) \overset{!}{=} 0 \nonumber \\
\Leftrightarrow &  (\eta + \kappa) \log p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) = -(\eta + \kappa + \lambda) +  g(\boldsymbol{\theta})  + \eta \log p_{\textrm{old}}(\boldsymbol{\theta}). \label{eq:marg:d1}
\end{align*}
Thus, the optimal $p_{\boldsymbol{\omega}}(\boldsymbol{\theta})$ given by

\begin{align}
p_{\boldsymbol{\omega}}(\boldsymbol{\theta}) &= \exp \left(- \dfrac{\eta + \kappa + \lambda}{\eta + \kappa} \right) \exp \left( \dfrac{g(\boldsymbol{\theta})}{\eta + \kappa} \right) p_{\textrm{old}}(\boldsymbol{\theta})^{\dfrac{\eta}{\eta + \kappa}} \nonumber \\
& \propto \exp \left( \dfrac{g(\boldsymbol{\theta})}{\eta + \kappa} \right) p_{\textrm{old}}(\boldsymbol{\theta})^{\dfrac{\eta}{\eta + \kappa}} \nonumber \\
& = \exp \left(\dfrac{\eta \log p_{\textrm{old}}(\boldsymbol{\theta}) + g(\boldsymbol{\theta})}{\eta + \kappa} \right)
\end{align}

### Code
The code below implements the MORE algorithm. The heavy lifting of the algorithm is done by the nlopt package, which
is used for the optimization of the convex dual of the MORE problem.
In essence, MORE is a stochastic search algorithm with built-in entropy control, and as its optimization problem is a
constrained optimization that aims to improve the search distribution while maintaining its entropy and not having it
move to far per iteration.
This behavior is explained in the trust region slides of Slide Set 6, as well as Slides 23ff and 31ff of Slide Set 8.

As the algorithm is not explcitly mentioned in the lecture, you will again **not** need to implement anything here.

In [ ]:
class MORE:
    def __init__(self, task_dimension: int, kl_bound: float):
        self.task_dimension = task_dimension
        self.kl_bound = kl_bound

        self.gamma = 0.99
        self.beta_0 = 0.1
        self.eta_0 = 10
        self.omega_0 = 10
        self.h_0 = -50  # minimum entropy of the search distribution

        self._default_eta = 10
        self._default_omega = 10
        self._grad_bound = 1e-5

        self.opt = self._get_dual_optimizer()

        # constant values
        self._log_2_pi_k = self.task_dimension * (np.log(2 * np.pi))
        self._entropy_const = self.task_dimension * (np.log(2 * np.pi) + 1)

        # cached values
        self.beta = None
        self._eta = 1
        self._omega = 1
        self._old_term = None
        self._old_dist = None
        self._current_model = None
        self._dual = np.inf
        self._grad = np.zeros(2)
        self.kl_divergence = np.inf
        self._new_entropy = np.inf
        self._new_mean = None
        self._new_cov = None

    def _get_dual_optimizer(self):
        """
        Setting up NLOpt optimizer for the dual optimization
        :return:
        """
        opt = nlopt.opt(nlopt.LD_LBFGS, 2)
        opt.set_lower_bounds((1e-20, 1e-20))
        opt.set_upper_bounds((np.inf, np.inf))
        opt.set_ftol_abs(1e-12)
        opt.set_ftol_rel(1e-12)
        opt.set_xtol_abs(1e-12)
        opt.set_xtol_rel(1e-12)
        opt.set_maxeval(1000)
        opt.set_maxtime(5 * 60 * 60)

        def opt_func(x, grad):
            dual_value = self.dual_and_grad(x, grad)
            if np.isinf(dual_value):
                opt.set_lower_bounds((float(x[0]), 1e-20))
            return float(dual_value.flatten())

        opt.set_min_objective(opt_func)
        return opt

    def step(self, old_dist, surrogate):
        """
        Given an old distribution and a model object, perform one MORE iteration
        :param old_dist: Distribution object
        :param surrogate: quadratic model object
        :return: new distribution parameters and success variables
        """
        self.beta = self.gamma * (old_dist.entropy - self.h_0) + self.h_0  # set entropy constraint

        self._old_term = old_dist.log_det + old_dist.mean.T @ old_dist.natural_mean
        self._old_dist = old_dist
        self._current_model = surrogate

        self.opt.set_lower_bounds((1e-20, 1e-20))
        eta, omega, success = self.dual_opt()

        if success:
            self.eta_0 = self._eta
            self.omega_0 = self._omega
            return self._new_mean, self._new_cov, True
        else:
            # in some cases, the optimization may fail due to numerical issues.
            # In this case, we use default values and try again in the next iteration.
            self.eta_0 = self._default_eta
            self.omega_0 = self._default_omega
            return old_dist.mean, old_dist.covariance, False

    def dual_opt(self):
        success_kl = False
        success_entropy = False
        try:
            eta, omega = self.opt.optimize(np.hstack([self.eta_0, self.omega_0]))
            opt_val = self.opt.last_optimum_value()
        except (RuntimeError, nlopt.ForcedStop, nlopt.RoundoffLimited) as e:
            eta = self._eta
            omega = self._omega
            opt_val = self._dual
        if ~np.isinf(opt_val):
            if self.kl_divergence < 1.1 * self.kl_bound:
                success_kl = True

            if self._new_entropy > 0:
                if self._new_entropy > 0.9 * self.beta:
                    success_entropy = True
            else:
                if self._new_entropy > 1.1 * self.beta:
                    success_entropy = True
        success = success_kl and success_entropy

        return eta, omega, success

    def dual_and_grad(self, x, grad):
        """
        This is where the magic happens! We build and solve the dual function using a lot of mathematical tricks.
        This function is used by the convex optimizer (NLOpt) to figure out the right values for the next update.
        While it may seem threatening, it closely follows the MORE derivations of the exercise above.
        :param x:
        :param grad:
        :return:
        """
        eta = x[0]
        omega = x[1]
        self._eta = eta
        self._omega = omega

        new_lin, new_prec = new_natural_params(eta, omega, self._old_dist, self._current_model)

        try:
            kl_bound = self.kl_bound
            beta = self.beta
            log_2_pi_k = self._log_2_pi_k
            old_term = self._old_term

            chol_new_precision = np.linalg.cholesky(new_prec)
            inv_chol_new_precision = np.linalg.inv(chol_new_precision)

            new_cov = inv_chol_new_precision.T @ inv_chol_new_precision
            new_mean = new_cov @ new_lin

            chol_new_covariance = np.linalg.cholesky(new_cov)
            # compute log(det(Sigma_pi))
            new_log_det = 2 * np.sum(np.log(np.diag(chol_new_covariance)))

            dual_value = eta * kl_bound - omega * beta + 0.5 * (omega * log_2_pi_k
                                                                - eta * old_term
                                                                + (eta + omega) * (new_log_det + new_mean.T @ new_lin)
                                                                )

            mahalanobis_distance = np.sum((self._old_dist.chol_prec.T @ (self._old_dist.mean - new_mean)) ** 2)
            trace_term = np.sum(np.square(self._old_dist.chol_prec.T @ chol_new_covariance))

            kl_divergence = 0.5 * (mahalanobis_distance
                                   + self._old_dist.log_det
                                   - new_log_det
                                   + trace_term
                                   - self.task_dimension)

            entropy = 0.5 * (new_log_det + self._log_2_pi_k + self.task_dimension)

            grad[0] = self.kl_bound - float(kl_divergence)
            grad[1] = entropy - self.beta

        except np.linalg.LinAlgError:
            # Skip the update if it fails due to numerical issues. We can then try again at the next iteration
            dual_value = np.atleast_1d(np.inf)
            kl_divergence = np.inf
            entropy = -np.inf
            new_mean = self._old_dist.mean
            new_cov = self._old_dist.covariance
            grad[0] = -0.1
            grad[1] = 0.

        self._dual = dual_value
        self.kl_divergence = kl_divergence
        self._new_entropy = entropy
        self._new_mean = new_mean
        self._new_cov = new_cov
        self._grad = grad
        return dual_value

# Running the algorithm

As usual, this last part of the exercise defines arguments/hyperparameters, as well as a general training loop. You do
*not* need to change any code here (unless you want to fiddle with the parameters). If everything is implemented
correctly, you should see training improvements almost immediately and convergence after a couple thousand steps.
The code will save
* a couple of training metrics,
* visualizations of $10$ random search distribution for the point reaching task, along with their reward evaluation

For the homework, you only need to send in the last visualization, i.e., the one at Iteration 4092, but you can also
turn in all of the plots as usual.

In [ ]:

###
class Args:

    # Boilerplate for properly accessing the args
    def __getitem__(self, key):
        return getattr(self, key)

    def __setitem__(self, key, val):
        setattr(self, key, val)

    num_links = 5  # @param {type: "integer"}
    num_iterations = 5000  # @param {type: "integer"}
    samples_per_iteration = 512  # @param {type: "integer"}
    kl_bound = 0.2 # @param {type: "number"}


def main(args: Args):
    np.random.seed(0)

    kl_bound = args.kl_bound
    num_links = args.num_links
    num_iterations = args.num_iterations
    samples_per_iteration = args.samples_per_iteration

    smoothness_prior = [1.0] + [0.04] * (num_links - 1)
    reacher = PointReacher(num_links=num_links, likelihood_std=1.0e-4,
                           smoothness_prior_std=smoothness_prior)

    search_dist = Gaussian(mean=np.zeros(num_links), covariance=smoothness_prior * np.eye(num_links))
    surrogate = QuadraticModel(dimension=num_links)
    more = MORE(task_dimension=num_links, kl_bound=kl_bound)

    progress_bar = ProgressBar(num_iterations=num_iterations)

    full_metrics = {"mean_reward": [],
                    "kl_change": [],
                    "entropy": []}

    for iteration in range(num_iterations):
        if iteration == 0 or 2 ** round(np.log2(iteration)) == iteration:
            # plot for each power of 2. This gives a lot of plots early on, when the distribution still changes
            # quickly, and eventually slows down when near convergence

            plot_samples = search_dist.sample(10)
            reacher.render(samples=plot_samples, iteration=iteration)
            save_figure(save_name=f"point_reacher_kl={kl_bound}_iteration={iteration:04d}")
            plot_metrics(full_metrics)

        samples = search_dist.sample(samples_per_iteration)

        rewards = reacher.reward(samples)

        success = surrogate.fit(samples, rewards)
        if not success:
            progress_bar.update(1)
            continue

        new_mean, new_cov, success = more.step(search_dist, surrogate)

        if success:
            try:
                search_dist.update_params(new_mean, new_cov)
            except Exception as e:
                print(e)
        mean_reward = reacher.reward(search_dist.mean.T)

        # logging utility
        progress_bar(mean_reward=mean_reward, kl_change=more.kl_divergence, entropy=search_dist.entropy)
        full_metrics["mean_reward"].append(mean_reward)
        full_metrics["kl_change"].append(more.kl_divergence)
        full_metrics["entropy"].append(search_dist.entropy)

args = Args()
main(args=args)
